In [1]:
%tensorflow_version 2.x

In [28]:
from __future__ import absolute_import, division, print_function, unicode_literals


import tensorflow as tf
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [5]:
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

In [3]:
train_path = tf.keras.utils.get_file(
    "iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file(
    "iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

16384/573 [=========================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================] - 0s 0us/step


In [18]:
train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)

In [12]:
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [19]:
train.head(1)

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2


In [20]:
train_y = train.pop('Species')
train_x = train
test_y = test.pop('Species')
test_x = test

In [21]:
train.keys()

Index(['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth'], dtype='object')

In [22]:
# Feature columns list
my_feature_columns = []
for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))
print(my_feature_columns)

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


In [23]:
# Input Function
def input_fn(features, labels, training=True, batch_size=256):
    # Convert the inputs to a Dataset object.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle and repeat if you are in training mode.
    if training:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)

In [ ]:
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    hidden_units=[30,10],
    n_classes = 3
)

In [ ]:
# training the classifier
classifier.train(
    input_fn = lambda:input_fn(train_x, train_y, training=True),
    steps=5000
)
# We include a lambda to avoid creating an inner function previously

In [33]:
# evaluation
results = classifier.evaluate(
    input_fn = lambda: input_fn(test_x, test_y, training=False)
)
print('\nTest accuracy: {accuracy:0.3f}\n'.format(**results))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2022-02-05T09:07:41
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp6kyyzz6v/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.79582s
INFO:tensorflow:Finished evaluation at 2022-02-05-09:07:42
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.9, average_loss = 0.53716654, global_step = 5000, loss = 0.53716654
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: /tmp/tmp6kyyzz6v/model.ckpt-5000

Test accuracy: 0.900



In [34]:
results

{'accuracy': 0.9,
 'average_loss': 0.53716654,
 'global_step': 5000,
 'loss': 0.53716654}